## TESTING ESDRx2 MODEL

In [1]:
import torch, cv2, os, time
from torchvision import transforms
from PIL import Image

In [2]:
model = torch.jit.load("./saved_models/EDSRx2v4.20.pt").cpu()
model.eval()

RecursiveScriptModule(
  original_name=EDSR
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (res_blocks): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=ResidualBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
    )
    (1): RecursiveScriptModule(
      original_name=ResidualBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
    )
    (2): RecursiveScriptModule(
      original_name=ResidualBlock
      (conv1): RecursiveScriptModule(original_name=Conv2d)
      (relu): RecursiveScriptModule(original_name=ReLU)
      (conv2): RecursiveScriptModule(original_name=Conv2d)
    )
    (3): RecursiveScriptModule(
      original_name=ResidualBlock
      (conv1): RecursiveScrip

In [3]:
def SuperSample(img=None, scale_factor=2):
    image_transform = transforms.Compose([
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    ])
    if img != None:
        # Load the input image
        input_image = Image.open(img).convert('RGB')
        input_image = image_transform(input_image).unsqueeze(0)  # Add a batch dimension
#         # Move the input image to the GPU if available
#         if torch.cuda.is_available():
#             input_image = input_image.cuda()
        input_image = input_image.cpu()
        # Perform image upscaling
        with torch.no_grad():
            print("Super Sampling Image...", end="\n")
            output_image = model(input_image)
        # Convert the output tensor to a PIL image
        output_image = output_image.squeeze(0).cpu().clamp(0, 1).numpy()
        output_image = (output_image.transpose(1, 2, 0) * 255).astype('uint8')
        output_image = Image.fromarray(output_image)
        output_image.save("temp.jpg")
        print("Post-Processing Image...")
        #post-processing
        kernel = (5, 5)
        output_image = cv2.imread("./temp.jpg")
        os.remove("temp.jpg")
        gauss_img = cv2.GaussianBlur(output_image, kernel, 0)
        lin_img = cv2.resize(
            gauss_img, (gauss_img.shape[1], gauss_img.shape[0]), interpolation=cv2.INTER_LINEAR
        )
        sharp_image = cv2.addWeighted(output_image, 1, gauss_img, -0.0002, 0)
        gauss_img = cv2.GaussianBlur(sharp_image, (3, 3), 0)
        cv2.imwrite('upscaled_image.jpg', output_image)
        print("Image Generated Successfully!")
        

In [12]:
start = time.time()
SuperSample("../train_LR/0001.png")
end = time.time()
print(f"Executed in {end - start}s")

Super Sampling Image...
Post-Processing Image...
Image Generated Successfully!
Executed in 2.1003267765045166s


In [10]:
def SuperSampleCV2(img=None):
    if img != None:
        img = cv2.imread(img)
        img = cv2.resize(img, (640, 640), interpolation=cv2.INTER_LINEAR)
        cv2.imwrite("CV2.jpg", img)

In [7]:
start = time.time()
SuperSampleCV2("../train_LR/0400.png")
end = time.time()
print(f"Executed in {end - start}s")

Executed in 0.008253097534179688s
